In [1]:
!pip uninstall -qqy jupyterlab  # Remove unused conflicting packages
!pip install -U -q "google-genai==1.7.0"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.7/144.7 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.9/100.9 kB 5.7 MB/s eta 0:00:00


In [2]:
import os
from kaggle_secrets import UserSecretsClient

GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

In [3]:
import os
import time
from google.generativeai import configure, GenerativeModel
from pypdf import PdfReader
from typing import Optional


In [4]:
# # Configure API
# configure(api_key=os.environ['GOOGLE_API_KEY'])  # Or your actual key

In [5]:
# Configure with the correct API endpoint
configure(
    api_key=os.environ["GOOGLE_API_KEY"],
    client_options={"api_endpoint": "generativelanguage.googleapis.com"}
)

In [6]:
class AOCAssistant:
    def __init__(self, pdf_path: str = "/kaggle/input/accounting-cheat-code/accounting cheat code.pdf"):
        """Initialize with proper model selection and rate control"""
        self.pdf_text = self._extract_pdf_text(pdf_path)
        
        # Try the latest model names
        try:
            self.model = GenerativeModel("gemini-1.5-pro-latest")  # Newest model
            print("Using Gemini 1.5 Pro")
        except:
            try:
                self.model = GenerativeModel("gemini-pro")  # Fallback to 1.0
                print("Using Gemini 1.0 Pro")
            except Exception as e:
                raise ValueError(f"Model initialization failed: {str(e)}")
        
        self.last_call_time = 0
        self.min_interval = 1  # 1 second between calls (more realistic than 20s)

    def _extract_pdf_text(self, path: str) -> str:
        """Robust PDF text extraction"""
        try:
            reader = PdfReader(path)
            text = []
            for page in reader.pages:
                page_text = page.extract_text() or ""
                text.append(page_text)
            return "\n".join(text)[:20000]  # Limit to first 20k characters
        except Exception as e:
            raise RuntimeError(f"PDF processing failed: {str(e)}")

    def query(self, question: str) -> str:
        """Improved query with better error handling"""
        # Rate limiting
        elapsed = time.time() - self.last_call_time
        if elapsed < self.min_interval:
            time.sleep(self.min_interval - elapsed)
        
        try:
            self.last_call_time = time.time()
            
            prompt = f"""You are an expert accounting assistant. Use this document context:
            {self.pdf_text}
            
            Question: {question}
            
            Guidelines:
            1. Answer based on the document when possible
            2. For calculations, provide standard formulas
            3. If unsure, say "Not explicitly covered in this document"
            """
            
            response = self.model.generate_content(prompt)
            return response.text
            
        except Exception as e:
            return f"Error: {str(e)}. Please try again."

# Test with proper error handling
try:
    assistant = AOCAssistant()
    
    questions = [
        "Define accounts receivable",
        "How to calculate straight-line depreciation?",
        "Define accounts payable"
          ]
    
    for q in questions:
        print(f"\n🔍 Question: {q}")
        answer = assistant.query(q)
        print(f"📝 Answer: {answer[:500]}{'...' if len(answer)>500 else ''}")
        
except Exception as e:
    print(f"Initialization error: {str(e)}")
    print("Troubleshooting steps:")
    print("1. Verify GOOGLE_API_KEY is set correctly")
    print("2. Check PDF exists at specified path")
    print("3. Ensure notebook has internet access")

Using Gemini 1.5 Pro

🔍 Question: Define accounts receivable
📝 Answer: While the provided cheat sheet mentions "accounts receivable" in several places (as an example of an asset, in the acid-test ratio, and regarding bad debts expense), it does *not* explicitly define the term.  It is implied to be an asset that will turn to cash within one year.


🔍 Question: How to calculate straight-line depreciation?
📝 Answer: This cheat sheet mentions depreciation and accumulated depreciation in the context of adjusting entries and contra accounts, but it doesn't explain how to calculate straight-line depreciation.  However, the general formula for straight-line depreciation is:

(Asset Cost - Salvage Value) / Useful Life = Depreciation Expense per Period


🔍 Question: Define accounts payable
📝 Answer: Accounts payable is an example of a liability.  Liabilities are defined in the document as obligations and deferred revenues. Specifically, accounts payable is an example of a short-term liability o